## Using neural networks to predict on Kryptonite-N dataset

In [1]:
import os
myseed = 6095
os.environ['PYTHONHASHSEED'] = str(myseed)
os.environ["CUBLAS_WORKSPACE_CONFIG"]= ":4096:8"
os.environ["PYTORCH_CUDA_ALLOC_CONF"]= "expandable_segments:True"
print(os.getenv("CUBLAS_WORKSPACE_CONFIG"))
print(os.getenv("PYTORCH_CUDA_ALLOC_CONF"))

import numpy as np
import random
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from itertools import product
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args
from skopt.plots import plot_convergence, plot_gaussian_process
import time
from codecarbon import EmissionsTracker


:4096:8
expandable_segments:True


KeyboardInterrupt: 

In [ ]:
print(torch.__version__)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device: {0}'.format(device))


def set_seeds(myseed):

    random.seed(myseed)
    np.random.seed(myseed)
    torch.manual_seed(myseed)
    torch.cuda.manual_seed(myseed)
    torch.cuda.manual_seed_all(myseed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True)


set_seeds(myseed)


2.5.1+cu118
Device: cuda


In [ ]:
import matplotlib.pyplot as plt


def plot_features_vs_label(X, y):

    n_samples, n_features = X.shape
    
    assert y.shape == (n_samples,)
    assert set(np.unique(y)).issubset({0, 1})
    
    for i in range(n_features):
        plt.figure()
        
        plt.scatter(X[y == 0, i], [0] * sum(y == 0), color='blue', label='Label 0', alpha=0.6)
        plt.scatter(X[y == 1, i], [1] * sum(y == 1), color='red', label='Label 1', alpha=0.6)
        
        plt.xlabel(f'Feature {i+1}')
        plt.ylabel('Label')
        plt.title(f'Feature {i+1} vs. Label')
        plt.legend()
        plt.grid(True)
        
        plt.show()


n = 15
X = np.load('Datasets/kryptonite-%s-X.npy'%(n))
y = np.load('Datasets/kryptonite-%s-y.npy'%(n))

# plot_features_vs_label(X, y)


In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, depth):
        super().__init__()
        # self.linear_layer_stack = nn.Sequential(
        #     nn.Linear(input_size, hidden_size),
        #     nn.ReLU(),
        #     nn.Linear(hidden_size,hidden_size), 
        #     nn.ReLU(),
        #     nn.Linear(hidden_size,1), 
        # )



        modules = [torch.nn.Linear(input_size,hidden_size), torch.nn.LeakyReLU()]

        for i in range(depth-1):
            modules.append(torch.nn.Linear(hidden_size,hidden_size))
            modules.append(torch.nn.LeakyReLU())

        self.output_layer = torch.nn.Linear(hidden_size, 1)

        self.hidden_layer_stack = torch.nn.Sequential(
            *modules,
        )
        
        set_seeds(myseed)
        self._initialize_weights()  


    def _initialize_weights(self):
        for layer in self.hidden_layer_stack:
            if isinstance(layer, nn.Linear):
                torch.nn.init.kaiming_uniform_(layer.weight)
                torch.nn.init.zeros_(layer.bias)


    def forward(self, x):
        x = self.hidden_layer_stack(x)
        return self.output_layer(x)

#### Validation

In [ ]:
def validate_nn(model, X_val, y_val):
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val.to(device))
        val_outputs = torch.round(torch.sigmoid(val_outputs)).cpu().numpy()
        accuracy = accuracy_score(y_val, val_outputs)
    return accuracy

In [ ]:



# train_size = len(X_train)
# def train_nn(model, dataloader, criterion, optimizer, epochs):

def train_nn(model, dataloader, criterion, optimizer, epochs, X_val, y_val):
    start= time.perf_counter()
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.unsqueeze(1).to(device)  # Move data to device
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
        
        print(f'Epoch {epoch+1}/{epochs}, Loss: {epoch_loss/len(dataloader):.4f}')
        # eoe_time = time.perf_counter()
        # if eoe_time-start > 150:
        #     return epoch+1
            
        
        
        if (epoch+1)%10 == 0:
            validation_acc = validate_nn(model,X_val, y_val)
            print(f"Validation Accuracy: {validation_acc}")

        # if epoch%10 == 0:
        #     validate_nn(model, )

    return epochs


        



#### Grid Search Hyperparameter Tuning

In [ ]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)


In [ ]:
def grid_search(X_train, y_train, X_val, y_val, param_grid, krypto_n):
    best_accuracy = 0
    best_params = None
    best_model = None
    
    for depth, hidden_size, learning_rate, batch_size, epochs in product(*param_grid.values()):


        print(f"Training with depth={depth}, hidden_size={hidden_size}, learning_rate={learning_rate}, batch_size={batch_size}, epochs={epochs}")
        
        model = NeuralNetwork(input_size=krypto_n, hidden_size=hidden_size, depth=depth).to(device)
        criterion = nn.BCEWithLogitsLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


        g = torch.Generator()
        g.manual_seed(myseed)
        
        train_dataset = TensorDataset(X_train, y_train)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, worker_init_fn=seed_worker, generator=g)
        
        train_nn(model, train_loader, criterion, optimizer, epochs=epochs)
        
        accuracy = validate_nn(model, X_val, y_val)
        print(f"Validation Accuracy: {accuracy:.4f}")
        
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = {'depth':depth, 'hidden_size': hidden_size, 'learning_rate': learning_rate, 
                           'batch_size': batch_size, 'epochs': epochs}
            best_model = model

        print(f"Current best accuracy: {best_accuracy:.4f}")

    
    print("Best Parameters:", best_params)
    print("Best Validation Accuracy:", best_accuracy)
    print(f"Krypto variant: {krypto_n}")
    
    
    return best_model, best_params

#### Random Grid Search

In [ ]:
def rand_grid_search(X_train, y_train, X_val, y_val, krypto_n):
    best_accuracy = 0
    best_params = None
    best_model = None
    
    combos = 0
    max_combos = 10

    while best_accuracy < 0.90:


        # hidden_sizes = param_grid['hidden_size']
        # learning_rates = param_grid~learning_rate']
        # batch_sizes = param_grid['batch_size']
        # epoch_list = param_grid['epochs']

        hidden_size = np.random.randint(10,4000)
        learning_rate = np.random.uniform(0.0003, 0.0045)
        batch_size = np.random.randint(64,6000)
        epochs = np.random.randint(15, 50)
        depth = np.random.randint(2,3)



        print(f"Training with depth={depth}, hidden_size={hidden_size}, learning_rate={learning_rate}, batch_size={batch_size}, epochs={epochs}")
        
        model = NeuralNetwork(input_size=krypto_n, hidden_size=hidden_size, depth=depth).to(device)
        criterion = nn.BCEWithLogitsLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        
        g = torch.Generator()
        g.manual_seed(myseed)
        
        train_dataset = TensorDataset(X_train, y_train)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, worker_init_fn=seed_worker, generator=g)
        
        train_nn(model, train_loader, criterion, optimizer, epochs=epochs)
        
        accuracy = validate_nn(model, X_val, y_val)
        print(f"Validation Accuracy: {accuracy:.4f}")
        
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = {'depth': [depth], 'hidden_size': [hidden_size], 'learning_rate': [learning_rate], 
                           'batch_size': [batch_size], 'epochs': [epochs]}
            best_model = model

        print(f"Current best accuracy: {best_accuracy:.4f}")
        combos += 1


    print("Best Parameters:", best_params)
    print("Best Validation Accuracy:", best_accuracy)
    print(f"Krypto variant: {krypto_n}")
    print(combos)
    return best_model, best_params

In [ ]:
print(torch.__version__)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = "cpu"
print('Device: {0}'.format(device))

# setting kryptonite dataset no.
n = 30

# Reading and normalising dataset features for efficient convergence
scaler = StandardScaler()

X = np.load('Datasets/kryptonite-%s-X.npy'%(n))
y = np.load('Datasets/kryptonite-%s-y.npy'%(n))

# print(f"X shape: {X.shape()}")
print(np.shape(X))

if n>18:
    X_add = np.load('Datasets/additional-kryptonite-%s-X.npy'%(n))
    print(np.shape(X_add))
    y_add = np.load('Datasets/additional-kryptonite-%s-y.npy'%(n))

    X=  np.concatenate((X,X_add),axis = 0)
    y=  np.concatenate((y,y_add),axis = 0)


print(np.shape(X))


# print((X[0]))

# # Shuffle and split the data
# X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.6, random_state=myseed)  # 60% training

# scaler.fit(X_train)
# scaler.transform(X_train)
# scaler.transform(X_temp)

# X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=myseed)  # 20% validation, 20% test

set_seeds(myseed)

# Shuffle and split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=myseed)  # 80% training-validation/20% testing

scaler.fit(X_train)
scaler.transform(X_train)
scaler.transform(X_test)

# X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=myseed)  # 20% validation, 20% test



X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32)
# X_val, y_val = torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.float32)
X_test, y_test = torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32)


print((X_train.size()))
# print((X_val[0]))
print((X_test[0]))



best_accuracy = 0


2.5.1+cu118
Device: cuda
(60000, 30)
(250000, 30)
(310000, 30)
torch.Size([248000, 30])
tensor([ 0.0209,  0.9877,  0.0376,  0.8977,  0.0126,  0.0118,  0.0183,  0.0038,
        -0.0080,  0.8416,  0.9639,  0.7767,  0.0628,  1.0071,  1.1124,  0.0173,
         1.0892, -0.0105,  0.0932,  0.9871,  0.9691,  1.0487,  0.0365,  0.0269,
         0.2208,  0.9698,  0.9534, -0.2391,  0.8032,  1.1243])


### Bayesian Hyperparameter Tuning

In [ ]:



# tracker = EmissionsTracker()
# tracker.start()

param_space = [
    Real(1e-5, 1e-1, prior="log-uniform", name="learning_rate"), 
    Integer(500, 2000, prior = "log-uniform",name="hidden_size"),              
    Real(2,2.5, prior="log-uniform" ,name="depth"),                           
    Integer(2000, 30000, prior = "log-uniform", name="batch_size"),                   
    Integer(10, 180,prior = "log-uniform", name="epochs"),
    Real(0.0000001,0.1, prior ="log-uniform", name="decay")
]

# n=15 paramspace
# param_space = [
#     Real(1e-5, 1e-2, prior="log-uniform", name="learning_rate"),  
#     Integer(2, 6000, prior = "log-uniform",name="hidden_size"),                       
#     Integer(2, 4, name="depth"),                               
#     Integer(16, 10000, prior = "log-uniform", name="batch_size"),                         
#     Integer(1, 500, name="epochs"),                             
# ]

# best validation performance at around:
# lr = 0.0002
# hidden_size = 889-1200
# depth = 2
# batch_size = 40
# epochs = 500


set_seeds(myseed)

krypto_n = n
k_folds = 5

@use_named_args(param_space)
# def objective(learning_rate, hidden_size, depth, batch_size, epochs):
#     torch.cuda.empty_cache()
#     model = NeuralNetwork(input_size=krypto_n, hidden_size=hidden_size, depth=depth).to(device)
#     criterion = nn.BCEWithLogitsLoss()
#     optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#     train_dataset = TensorDataset(X_train, y_train)
#     g = torch.Generator()
#     g.manual_seed(myseed) 
#     train_loader = DataLoader(train_dataset, batch_size=int(batch_size), shuffle=True, worker_init_fn=seed_worker, generator=g, pin_memory=True)

#     print(f"Training with hyperparameters:\n\nlearning rate={learning_rate}\nhidden size={hidden_size}\ndepth={depth}\nbatch_size={batch_size}\nepochs={epochs}\n")
#     train_nn(model, train_loader, criterion, optimizer, epochs=epochs, X_val= X_val, y_val = y_val)
    
#     accuracy = validate_nn(model, X_val, y_val)

#     print(f"\nfor hyperparameters:\nlearning rate={learning_rate}\nhidden size={hidden_size}\ndepth={depth}\nbatch_size={batch_size}\nepochs={epochs}\n\n################        Final Validation Accuracy:     {accuracy:.4f}          ##################\n")
#     return -accuracy

def objective(learning_rate, hidden_size, depth, batch_size, epochs, decay):
    kf = KFold(n_splits=k_folds, shuffle=True, random_state=myseed)
    fold_accuracies = []

    print(f"Training with hyperparameters:\n\nlearning rate={learning_rate}\nhidden size={hidden_size}\ndepth={np.floor(depth)}\nbatch_size={batch_size}\nepochs={epochs}\ndecay={decay}\n")
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_train)):
        # fold_start = time.perf_counter()
        print(f"Fold {fold + 1}/{k_folds}")

        X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
        y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]
        


        model = NeuralNetwork(input_size=krypto_n, hidden_size=hidden_size, depth=int(np.floor(depth))).to(device)
        criterion = nn.BCEWithLogitsLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=decay)
        
        train_dataset = TensorDataset(X_train_fold, y_train_fold)
        g = torch.Generator()
        g.manual_seed(myseed) 
        train_loader = DataLoader(train_dataset, batch_size=int(batch_size), shuffle=True, worker_init_fn=seed_worker, generator=g)
        
        epochs = train_nn(model, train_loader, criterion, optimizer, epochs=epochs, X_val =X_val_fold, y_val = y_val_fold)
        # epochs = train_nn(model, train_loader, criterion, optimizer, epochs=epochs)
       
       
        torch.cuda.empty_cache()


        print(f"\nTrained in {epochs} epochs\n")
        
        # fold_end = time.perf_counter()

        accuracy = validate_nn(model, X_val_fold, y_val_fold)
        print(f"Validation Accuracy for Fold {fold + 1}: {accuracy:.4f}")
        print(f"\nwith hyperparameters:\n\nlearning rate={learning_rate}\nhidden size={hidden_size}\ndepth={int(np.floor(depth))}\nbatch_size={batch_size}\nepochs={epochs}\ndecay={decay}\n\n\n")
        if accuracy < 0.74:
            return -accuracy
        
        # fold_duration = fold_end - fold_start
        # if fold_duration > 80:
            # return 1
        
        fold_accuracies.append(accuracy)
    
    avg_accuracy = sum(fold_accuracies) / len(fold_accuracies)
    print(f"Average Validation Accuracy: {avg_accuracy:.4f} for hyperparameters:\n\nlearning rate={learning_rate}\nhidden size={hidden_size}\ndepth={np.floor(depth)}\nbatch_size={batch_size}\nepochs={epochs}\ndecay={decay}\n\n\n")


    # min_accuracy = min(fold_accuracies)
    # print(f"Min Validation Accuracy: {min_accuracy:.4f} for hyperparameters:\n\nlearning rate={learning_rate}\nhidden size={hidden_size}\ndepth=2\nbatch_size={batch_size}\nepochs={epochs}\n\n\n")    

    return -avg_accuracy



result = gp_minimize(objective, param_space, n_calls=60, random_state=myseed)

plot_convergence(result)

best_params = result.x

# emissions: float = tracker.stop()
# print(emissions)



Training with hyperparameters:

learning rate=0.0015341702952119374
hidden size=851
depth=2.0
batch_size=28553
epochs=26
decay=6.5814914921113845e-06

Fold 1/5


e:\materials\Year 4\autumn\MML\ImpCML\.venv\Lib\site-packages\torch\nn\modules\module.py:1326: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  return t.to(


Epoch 1/26, Loss: 1.1054
Epoch 2/26, Loss: 0.7529
Epoch 3/26, Loss: 0.7021
Epoch 4/26, Loss: 0.7046
Epoch 5/26, Loss: 0.6943
Epoch 6/26, Loss: 0.6950
Epoch 7/26, Loss: 0.6936
Epoch 8/26, Loss: 0.6933
Epoch 9/26, Loss: 0.6933
Epoch 10/26, Loss: 0.6931
Validation Accuracy: 0.4994556451612903
Epoch 11/26, Loss: 0.6931
Epoch 12/26, Loss: 0.6930
Epoch 13/26, Loss: 0.6929
Epoch 14/26, Loss: 0.6929
Epoch 15/26, Loss: 0.6928
Epoch 16/26, Loss: 0.6928
Epoch 17/26, Loss: 0.6927
Epoch 18/26, Loss: 0.6927
Epoch 19/26, Loss: 0.6926
Epoch 20/26, Loss: 0.6925
Validation Accuracy: 0.4993951612903226
Epoch 21/26, Loss: 0.6924
Epoch 22/26, Loss: 0.6924
Epoch 23/26, Loss: 0.6922
Epoch 24/26, Loss: 0.6921
Epoch 25/26, Loss: 0.6920
Epoch 26/26, Loss: 0.6918

Trained in 26 epochs

Validation Accuracy for Fold 1: 0.5011

with hyperparameters:

learning rate=0.0015341702952119374
hidden size=851
depth=2
batch_size=28553
epochs=26
decay=6.5814914921113845e-06



Training with hyperparameters:

learning rate=3.

OutOfMemoryError: CUDA out of memory. Tried to allocate 670.00 MiB. GPU 0 has a total capacity of 6.00 GiB of which 0 bytes is free. Of the allocated memory 4.09 GiB is allocated by PyTorch, and 698.61 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
set_seeds(myseed)

model = NeuralNetwork(input_size=krypto_n, hidden_size=best_params[1], depth=int(best_params[2])).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=best_params[0], weight_decay=best_params[5])

train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=int(best_params[3]), shuffle=True)



# train_nn(model, train_loader, criterion, optimizer, epochs=best_params[4])
train_nn(model, train_loader, criterion, optimizer, epochs=best_params[4], X_val = X_test, y_val=y_test)

test_accuracy = validate_nn(model, X_test, y_test)
print(f"Test Accuracy with Best Hyperparameters: {test_accuracy:.4f}")


# set_seeds(myseed)
# model = NeuralNetwork(input_size=n, hidden_size =514, depth=2).to(device)
# criterion = nn.BCEWithLogitsLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.0008212520954386176, weight_decay=0.0006296438079895955)

# train_dataset = TensorDataset(X_train, y_train)
# train_loader = DataLoader(train_dataset, batch_size=int(84), shuffle=True)



# # train_nn(model, train_loader, criterion, optimizer, epochs=best_params[4])
# train_nn(model, train_loader, criterion, optimizer, epochs=500, X_val = X_test, y_val=y_test)

# test_accuracy = validate_nn(model, X_test, y_test)
# print(f"Test Accuracy with Best Hyperparameters: {test_accuracy:.4f}")

# n=9
# seed = 6095
# 80/20
# Test Accuracy with Best Hyperparameters: 0.9551

# 70/30

# 90/10  



# seed = 42
# Test Accuracy with Best Hyperparameters: 0.9595

# seed = 81
# Test Accuracy with Best Hyperparameters: 0.9574

# seed = 180
# Test Accuracy with Best Hyperparameters: 0.9571

# seed = 4
# Test Accuracy with Best Hyperparameters: 0.9570


# n=12
# seed = 6095
# Test Accuracy with Best Hyperparameters: 0.9324

# seed = 42
# Test Accuracy with Best Hyperparameters: 0.9457

# seed = 81
# Test Accuracy with Best Hyperparameters: 0.9484

# seed = 180
# Test Accuracy with Best Hyperparameters: 0.9315

# seed = 4
# Test Accuracy with Best Hyperparameters: 0.9505



# n=18 Test Accuracy with Best Hyperparameters: 0.8527


# n=12
# Test Accuracy with Best Hyperparameters: 0.9356


# n=15 

# [0.0008212520954386176, np.int64(514), np.int64(2), np.int64(84), np.int64(500), 0.0006296438079895955]


# seed = 6095
# Test Accuracy with Best Hyperparameters: 0.9408

# seed = 42
# Test Accuracy with Best Hyperparameters: 0.9466

# seed = 81
# Test Accuracy with Best Hyperparameters: 0.9420

# seed = 180
# Test Accuracy with Best Hyperparameters: 0.9379

# seed = 4
# Test Accuracy with Best Hyperparameters: 0.9380




# n=18 

# [0.005000499108369632, np.int64(1000), 2.3181051861840816, np.int64(1778), np.int64(264), 0.00036096505804747496]

# seed = 6095
# Test Accuracy with Best Hyperparameters: 0.9058

# seed = 42

# seed = 81

# seed = 180

# seed = 4



# n=24
# Test Accuracy with Best Hyperparameters: 0.8631



# n=30 
# 



Epoch 1/86, Loss: 0.8068
Epoch 2/86, Loss: 0.6932
Epoch 3/86, Loss: 0.6932
Epoch 4/86, Loss: 0.6932
Epoch 5/86, Loss: 0.6931


KeyboardInterrupt: 

In [ ]:
print(best_params)
# plot_gaussian_process(result)


# n = 15: best params = [0.0008422357289338771, np.int64(429), np.int64(2), np.int64(748), np.int64(79)]
# n = 18: [0.0003161682862801257, np.int64(829), np.int64(2), np.int64(132), np.int64(500)]


# 5-fold cross val
# n = 9: 
# [6.892793602028987e-05, np.int64(160), np.int64(5), np.int64(238), np.int64(85)]

# [0.0006529817319234669, np.int64(589), np.int64(2), np.int64(2247), np.int64(500), 0.0009065011800271416]



# n=12
# [9.850645640011949e-05, np.int64(413), np.int64(3), np.int64(975), np.int64(320)]

# [0.002922344328571065, np.int64(55), np.int64(2), np.int64(53), np.int64(299), 0.0004494118722718984]


# n=15
# [0.0003341615530109006, np.int64(889), np.int64(2), np.int64(61), np.int64(382)]
# [0.0004385205351420517, np.int64(966), np.int64(2), np.int64(334), np.int64(350), 0.0011896621969900364]
# [0.0004331825561706844, np.int64(3000), np.int64(2), np.int64(55), np.int64(350), 0.0005157392861889768]
# [5.123356347896232e-05, np.int64(2129), np.int64(2), np.int64(171), np.int64(350), 0.00042419048245948736]
# [0.0013701186872623389, np.int64(3000), np.int64(2), np.int64(56), np.int64(231), 0.00010044878532667275]

# [0.0008212520954386176, np.int64(514), np.int64(2), np.int64(84), np.int64(500), 0.0006296438079895955]

# n=18:
# [0.005000499108369632, np.int64(1000), 2.3181051861840816, np.int64(1778), np.int64(290), 0.00036096505804747496]


# n=24:
# [0.008349349990043525, np.int64(800), 2.7541886562632367, np.int64(2000), np.int64(86), 1e-05]


[0.008349349990043525, np.int64(800), 2.7541886562632367, np.int64(2000), np.int64(86), 1e-05]


In [ ]:

# param_grid = {
#     'depth': [2,4,8,16],
#     'hidden_size': [8, 16, 24, 30], 
#     'learning_rate': [0.001, 0.005, 0.01, 0.05 , 0.1], 
#     'batch_size': [16, 32, 64, 96],     
#     'epochs': [5, 10, 20, 30]              
# }

# Dictionary of hyperparameter values to search through

# param_grid =  {'depth': [2], 'hidden_size': [1295], 'learning_rate': [0.002355056560959035], 'batch_size': [2469], 'epochs': [38]}


# # print("Random Search")
# # best_model, best_params = rand_grid_search(X_train, y_train, X_val, y_val, n)

# set_seeds(myseed)

# print("Reproducibility check")
# best_model, best_params = grid_search(X_train, y_train, X_val, y_val, param_grid, n)


# Current Hyperparameters for each Kryptonite Variant:

# Best Parameters: {'depth': 3, 'hidden_size': 38, 'learning_rate': 0.00141, 'batch_size': 140, 'epochs': 35}
# Best Validation Accuracy: 0.9527777777777777
# Best Validation Accuracy: 0.955
# Best Validation Accuracy: 0.9546296296296296
# Best Validation Accuracy: 0.9512962962962963
# Best Validation Accuracy: 0.9555555555555556
# Best Validation Accuracy: 0.9535185185185185




# Test Accuracy = 0.9566666666666667

# Krypto variant: 9


# Best Parameters: {'depth': 2, 'hidden_size': 84, 'learning_rate': 0.002226459276380165, 'batch_size': 155, 'epochs': 146}
# Best Validation Accuracy: 0.9452777777777778
# Test Accuracy = 0.9497222222222222
# Krypto variant: 12


# Best Parameters: {'depth': 2, 'hidden_size': 54, 'learning_rate': 0.002077090842646019, 'batch_size': 147, 'epochs': 165}
# Best Validation Accuracy: 0.9342222222222222
# Test Accuracy = 0.9304444444444444
# Krypto variant: 15

# Current best accuracy: 0.9048
# Best Parameters: {'depth': 2, 'hidden_size': 97, 'learning_rate': 0.002937306209994177, 'batch_size': 100, 'epochs': 72}
# Best Validation Accuracy: 0.9048148148148148
# Test Accuracy = 0.9057407407407407
# Krypto variant: 18

# Training with depth=2, hidden_size=84, learning_rate=0.0008952525037794653, batch_size=155, epochs=146
# Validation Accuracy: 0.9427
# Krypto variant: 15

# Best Parameters: {'depth': 2, 'hidden_size': 150, 'learning_rate': 0.0007625120811532073, 'batch_size': 443, 'epochs': 45}
# Best Validation Accuracy: 0.86016
# Test Accuracy = 0.8596933333333333
# Krypto variant: 24

# Best Parameters: {'depth': 2, 'hidden_size': 388, 'learning_rate': 0.000522646472592812, 'batch_size': 485, 'epochs': 156}
# Best Validation Accuracy: 0.9032266666666666
# Best Validation Accuracy: 0.8842933333333334
# Best Validation Accuracy: 0.81512
# Best Validation Accuracy: 0.8572266666666667
# Best Validation Accuracy: 0.81512
# Best Validation Accuracy: 0.81512
# Best Validation Accuracy: 0.9123066666666667
# Best Validation Accuracy: 0.84404


# Krypto variant: 24

# NEW DATA:
# Training with depth=2, hidden_size=685, learning_rate=0.0026145410706330304, batch_size=4500, epochs=51
# Validation Accuracy: 0.8118



# Best Validation Accuracy: 0.5041834451901566
# Best Validation Accuracy: 0.7748322147651007
# Best Validation Accuracy: 0.8106263982102908
# Best Validation Accuracy: 0.7934451901565995
# Best Validation Accuracy: 0.8178635346756152
# Best Validation Accuracy: 0.7923266219239373
# Best Validation Accuracy: 0.7840268456375838
# Best Validation Accuracy: 0.7840268456375838
# Best Validation Accuracy: 0.7840268456375838
# Best Validation Accuracy: 0.7840268456375838
 
# Training with depth=2, hidden_size=1805, learning_rate=0.0026249175734212675, batch_size=4445, epochs=44

# 0.8185


# Best Validation Accuracy: 0.7513758389261745
# Best Validation Accuracy: 0.7513758389261745



# Best Parameters: {'depth': 2, 'hidden_size': 951, 'learning_rate': 0.0018208178202594196, 'batch_size': 2989, 'epochs': 30}
# Best Validation Accuracy: 0.7789597315436242

# Best Parameters: {'depth': 2, 'hidden_size': 951, 'learning_rate': 0.0018208178202594196, 'batch_size': 2989, 'epochs': 30}
# Best Validation Accuracy: 0.8215883668903803

# Best Parameters: {'depth': 2, 'hidden_size': 951, 'learning_rate': 0.0018208178202594196, 'batch_size': 2989, 'epochs': 30}
# Best Validation Accuracy: 0.8215883668903803

# Best Parameters: {'depth': 2, 'hidden_size': 951, 'learning_rate': 0.0018208178202594196, 'batch_size': 2989, 'epochs': 30}
# Best Validation Accuracy: 0.8215883668903803

# Best Parameters: {'depth': 2, 'hidden_size': 951, 'learning_rate': 0.0018208178202594196, 'batch_size': 2989, 'epochs': 30}
# Best Validation Accuracy: 0.789317673378076
 



# Training with depth=2, hidden_size=1295, learning_rate=0.002355056560959035, batch_size=2469, epochs=38
# Current best accuracy: 0.8386


In [ ]:
def test_nn(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        test_outputs = model(X_test.to(device))
        test_outputs = torch.round(torch.sigmoid(test_outputs)).cpu().numpy()
        accuracy = accuracy_score(y_test, test_outputs)
    return accuracy


test_accuracy = test_nn(best_model, X_test, y_test)
print(f"Test Accuracy = {test_accuracy}")

NameError: name 'best_model' is not defined